In [1]:
import pandas as pd
import helium
import csv

In [2]:
_PATH_ = "https://public.minsky.cc/maria/market_channel_factors_db"

In [3]:
df = pd.read_csv(_PATH_ + "/" + "databases/2019_03PreModelDatabaseOfWhitePotato.csv")

In [4]:
capitalOfRegions = pd.read_csv(_PATH_ + "/" + "databases/ProvinceCapitalOfRegions.csv")

In [5]:
capitalOfRegions[capitalOfRegions["NOMBREDD"] == "APURIMAC"] = "APURÍMAC"
capitalOfRegions[capitalOfRegions["NOMBREDD"] == "HUANUCO"] = "HUÁNUCO"
capitalOfRegions[capitalOfRegions["NOMBREDD"] == "JUNIN"] = "JUNÍN"

In [6]:
capitalOfRegions

,NOMBREPV,NOMBREDD
0,CHACHAPOYAS,AMAZONAS
1,HUARAZ,ANCASH
2,APURÍMAC,APURÍMAC
3,AREQUIPA,AREQUIPA
4,AYACUCHO,AYACUCHO
5,CAJAMARCA,CAJAMARCA
6,CUSCO,CUSCO
7,HUANCAVELICA,HUANCAVELICA
8,HUÁNUCO,HUÁNUCO
9,ICA,ICA


In [7]:
df.columns

Index(['Unnamed: 0', 'NOMBREDD', 'NOMBREPV', 'NOMBREDI', 'CONGLOMERADO',
       'NSELUA', 'UA', 'crop', 'production', 'totalProduction',
       'percentProduction', 'soldProduction', 'totalSoldProduction',
       'percentSoldProduction', 'farmGatePricePerKg', 'farmGateIncomes',
       'totalFarmGateIncomes', 'percentFarmGateIncomes', 'plantedArea',
       'totalPlantedArea', 'percentPlantedArea', 'yield', 'volumeOfDistrict',
       'associative', 'informalPriceInformation', 'formalPriceInformation',
       'informalMetereologicalInformation', 'formalMetereologicalInformation',
       'localMarket', 'regionalMarket', 'limaMarket', 'agroindustryMarket',
       'exportMarket', 'MARKET_NAN', 'MARKETS', 'MARKETS_QTY', 'marketSize',
       'collector', 'wholesaler', 'retailer', 'enterprise', 'finalConsumer',
       'marketChannelDirectedness', 'pointsOfSale'],
      dtype='object')

In [25]:
def createNumberTimeVariable(df, variableName, newVariableName):
    df[newVariableName] = df[variableName].apply(lambda x: x.split(" ")).apply(lambda x: '{0:.3g}'.format(int(x[0]) + int(x[2])/60) if (len(x) > 2) else '{0:.2g}'.format(int(x[0])/60) if x[1] == "min" else int(x[0]))

# District time to Lima Market

In [8]:
places = df["NOMBREDI"] + ", " + df["NOMBREPV"] + ", " + df["NOMBREDD"]
places = places.unique()

distances_dtlm = {}
times_dtlm = {}

helium.start_chrome()
for place in places:
    fixed_place = place.lower().replace(" ", "+")
    url = f"http://google.com/maps/dir/{fixed_place}/Mercado+Productores,+Santa+Anita"
    helium.go_to(url)
    q = helium.S("div.section-directions-trip-numbers")
    box = helium.find_all(q)
    if len(box) == 0:
        distances_dtlm[place] = None
        times_dtlm[place] = None
        continue
    dist_and_time = box[0]
    dist_and_time = dist_and_time.web_element.text
    
    d_t = dist_and_time.split("\n")
    times_dtlm[place] = d_t[0]
    distances_dtlm[place] = d_t[1]

In [9]:
with open("2019_04RoadTransportationDistrictToLimaMarket.csv", "w") as df:
    df.write("NOMBREDI,NOMBREPV,NOMBREDD,DistanceInTime,DistanceInKm\n")
    for key in distances_dtlm.keys():
        df.write("%s,%s,%s\n"%(key.replace(", ", ","), times_dtlm[key], distances_dtlm[key]))

In [8]:
districtToLimaMarket = pd.read_csv('2019_04RoadTransportationDistrictToLimaMarket.csv')

In [9]:
createNumberTimeVariable(districtToLimaMarket, "DistanceInTime", "districtTimeToLimaMarket")

NameError: name 'createNumberTimeVariable' is not defined

In [13]:
districtToLimaMarket

,NOMBREDI,NOMBREPV,NOMBREDD,DistanceInTime,DistanceInKm,districtTimeToLimaMarket
0,FLORIDA,BONGARA,AMAZONAS,20 h 3 min,1220 km,20.1
1,CHUQUIBAMBA,CHACHAPOYAS,AMAZONAS,19 h 35 min,1079 km,19.6
2,LA JALCA,CHACHAPOYAS,AMAZONAS,21 h 1 min,1139 km,21
3,LEVANTO,CHACHAPOYAS,AMAZONAS,20 h 51 min,1253 km,20.9
4,SOLOCO,CHACHAPOYAS,AMAZONAS,20 h 49 min,1259 km,20.8
...,...,...,...,...,...,...
383,POCOLLAY,TACNA,TACNA,17 h 51 min,1225 km,17.9
384,TACNA,TACNA,TACNA,17 h 46 min,1222 km,17.8
385,SITAJARA,TARATA,TACNA,19 h 28 min,1286 km,19.5
386,TARATA,TARATA,TACNA,19 h 28 min,1286 km,19.5


In [15]:
districtToLimaMarket.to_csv('2019_04RoadTransportationDistrictToLimaMarket.csv')


# District time to province capital of Region

In [10]:
def getCapitalProvinceDestination(region):
    row_selected = capitalOfRegions[capitalOfRegions["NOMBREDD"] == region].reset_index()
    print(region, "row_selected", row_selected, row_selected["NOMBREPV"], row_selected["NOMBREDD"])
    return (row_selected["NOMBREPV"] +  ", " + row_selected["NOMBREDD"])[0]

In [11]:
region = "LA LIBERTAD"
getCapitalProvinceDestination(region)

LA LIBERTAD row_selected    index  NOMBREPV     NOMBREDD
0     11  TRUJILLO  LA LIBERTAD 0    TRUJILLO
Name: NOMBREPV, dtype: object 0    LA LIBERTAD
Name: NOMBREDD, dtype: object


'TRUJILLO, LA LIBERTAD'

In [12]:
capitalOfRegions

,NOMBREPV,NOMBREDD
0,CHACHAPOYAS,AMAZONAS
1,HUARAZ,ANCASH
2,APURÍMAC,APURÍMAC
3,AREQUIPA,AREQUIPA
4,AYACUCHO,AYACUCHO
5,CAJAMARCA,CAJAMARCA
6,CUSCO,CUSCO
7,HUANCAVELICA,HUANCAVELICA
8,HUÁNUCO,HUÁNUCO
9,ICA,ICA


In [44]:
places = df["NOMBREDI"] + ", " + df["NOMBREPV"] + ", " + df["NOMBREDD"]
places = places.unique()

distances_dtpc = {}
times_dtpc = {}

helium.start_chrome()
for place in places:
    region = place.split(", ")[-1]
    initial_place = place.lower().replace(" ", "+")
    destination_place = getCapitalProvinceDestination(region).lower().replace(" ", "+")
    print(destination_place)
    url = f"http://google.com/maps/dir/{initial_place}/{destination_place}"
    helium.go_to(url)
    q = helium.S("div.section-directions-trip-numbers")
    box = helium.find_all(q)
    if len(box) == 0:
        distances_dtpc[place] = None
        times_dtpc[place] = None
        continue
    dist_and_time = box[0]
    dist_and_time = dist_and_time.web_element.text
    
    d_t = dist_and_time.split("\n")
    times_dtpc[place] = d_t[0]
    distances_dtpc[place] = d_t[1]


AMAZONAS row_selected    index     NOMBREPV  NOMBREDD
0      0  CHACHAPOYAS  AMAZONAS 0    CHACHAPOYAS
Name: NOMBREPV, dtype: object 0    AMAZONAS
Name: NOMBREDD, dtype: object
chachapoyas,+amazonas
AMAZONAS row_selected    index     NOMBREPV  NOMBREDD
0      0  CHACHAPOYAS  AMAZONAS 0    CHACHAPOYAS
Name: NOMBREPV, dtype: object 0    AMAZONAS
Name: NOMBREDD, dtype: object
chachapoyas,+amazonas
AMAZONAS row_selected    index     NOMBREPV  NOMBREDD
0      0  CHACHAPOYAS  AMAZONAS 0    CHACHAPOYAS
Name: NOMBREPV, dtype: object 0    AMAZONAS
Name: NOMBREDD, dtype: object
chachapoyas,+amazonas
AMAZONAS row_selected    index     NOMBREPV  NOMBREDD
0      0  CHACHAPOYAS  AMAZONAS 0    CHACHAPOYAS
Name: NOMBREPV, dtype: object 0    AMAZONAS
Name: NOMBREDD, dtype: object
chachapoyas,+amazonas
AMAZONAS row_selected    index     NOMBREPV  NOMBREDD
0      0  CHACHAPOYAS  AMAZONAS 0    CHACHAPOYAS
Name: NOMBREPV, dtype: object 0    AMAZONAS
Name: NOMBREDD, dtype: object
chachapoyas,+amazonas
AMAZO

IndexError: index out of bounds

In [71]:
with open("2019_04RoadTransportationDistrictToProvinceCapitalOfRegion.csv", "w") as df:
    df.write("NOMBREDI,NOMBREPV,NOMBREDD,DistanceInTime,DistanceInKm\n")
    for key in distances_dtpc.keys():
        df.write("%s,%s,%s\n"%(key.replace(", ", ","), times_dtpc[key], distances_dtpc[key]))

In [13]:
districtToProvinceCapitalOfRegion = pd.read_csv('2019_04RoadTransportationDistrictToProvinceCapitalOfRegion.csv')

In [14]:
districtToProvinceCapitalOfRegion.head()

,NOMBREDI,NOMBREPV,NOMBREDD,DistanceInTime,DistanceInKm
0,FLORIDA,BONGARA,AMAZONAS,2 h 12 min,95.3 km
1,CHUQUIBAMBA,CHACHAPOYAS,AMAZONAS,6 h 17 min,232 km
2,LA JALCA,CHACHAPOYAS,AMAZONAS,1 h 47 min,71.1 km
3,LEVANTO,CHACHAPOYAS,AMAZONAS,1 h,23.1 km
4,SOLOCO,CHACHAPOYAS,AMAZONAS,54 min,27.9 km


In [17]:
createNumberTimeVariable(districtToProvinceCapitalOfRegion, "DistanceInTime", "districtTimeToProvinceCapitalOfRegion")

In [18]:
districtToProvinceCapitalOfRegion

,NOMBREDI,NOMBREPV,NOMBREDD,DistanceInTime,DistanceInKm,districtTimeToProvinceCapitalOfRegion
0,FLORIDA,BONGARA,AMAZONAS,2 h 12 min,95.3 km,2.2
1,CHUQUIBAMBA,CHACHAPOYAS,AMAZONAS,6 h 17 min,232 km,6.28
2,LA JALCA,CHACHAPOYAS,AMAZONAS,1 h 47 min,71.1 km,1.78
3,LEVANTO,CHACHAPOYAS,AMAZONAS,1 h,23.1 km,1
4,SOLOCO,CHACHAPOYAS,AMAZONAS,54 min,27.9 km,0.9
...,...,...,...,...,...,...
383,POCOLLAY,TACNA,TACNA,7 min,3.4 km,0.12
384,TACNA,TACNA,TACNA,0 min,0 km,0
385,SITAJARA,TARATA,TACNA,1 h 35 min,88.1 km,1.58
386,TARATA,TARATA,TACNA,1 h 35 min,88.1 km,1.58


In [19]:
districtToProvinceCapitalOfRegion.to_csv('2019_04RoadTransportationDistrictToProvinceCapitalOfRegion.csv')

# Province capital time to Lima Market

In [20]:
places = capitalOfRegions["NOMBREPV"] + ", " + capitalOfRegions["NOMBREDD"]
places = places.unique()

distances_pctlm = {}
times_pctlm = {}

helium.start_chrome()
for place in places:
    region = place.split(", ")[-1]
    fixed_place = getCapitalProvinceDestination(region).lower().replace(" ", "+")
    url = f"http://google.com/maps/dir/{fixed_place}/Mercado+Productores,+Santa+Anita"
    helium.go_to(url)
    q = helium.S("div.section-directions-trip-numbers")
    box = helium.find_all(q)
    if len(box) == 0:
        distances_pctlm[place] = None
        times_pctlm[place] = None
        continue
    dist_and_time = box[0]
    dist_and_time = dist_and_time.web_element.text
    
    d_t = dist_and_time.split("\n")
    times_pctlm[place] = d_t[0]
    distances_pctlm[place] = d_t[1]
    


AMAZONAS row_selected    index     NOMBREPV  NOMBREDD
0      0  CHACHAPOYAS  AMAZONAS 0    CHACHAPOYAS
Name: NOMBREPV, dtype: object 0    AMAZONAS
Name: NOMBREDD, dtype: object
ANCASH row_selected    index NOMBREPV NOMBREDD
0      1   HUARAZ   ANCASH 0    HUARAZ
Name: NOMBREPV, dtype: object 0    ANCASH
Name: NOMBREDD, dtype: object
APURÍMAC row_selected    index  NOMBREPV  NOMBREDD
0      2  APURÍMAC  APURÍMAC 0    APURÍMAC
Name: NOMBREPV, dtype: object 0    APURÍMAC
Name: NOMBREDD, dtype: object
AREQUIPA row_selected    index  NOMBREPV  NOMBREDD
0      3  AREQUIPA  AREQUIPA 0    AREQUIPA
Name: NOMBREPV, dtype: object 0    AREQUIPA
Name: NOMBREDD, dtype: object
AYACUCHO row_selected    index  NOMBREPV  NOMBREDD
0      4  AYACUCHO  AYACUCHO 0    AYACUCHO
Name: NOMBREPV, dtype: object 0    AYACUCHO
Name: NOMBREDD, dtype: object
CAJAMARCA row_selected    index   NOMBREPV   NOMBREDD
0      5  CAJAMARCA  CAJAMARCA 0    CAJAMARCA
Name: NOMBREPV, dtype: object 0    CAJAMARCA
Name: NOMBREDD, 

In [21]:
with open("2019_04RoadTransportationProvinceCapitalToLimaMarket.csv", "w") as df:
    df.write("NOMBREPV,NOMBREDD,DistanceInTime,DistanceInKm\n")
    for key in distances_pctlm.keys():
        df.write("%s,%s,%s\n"%(key.replace(", ", ","), times_pctlm[key], distances_pctlm[key]))

In [26]:
provinceCapitalToLimaMarket = pd.read_csv('2019_04RoadTransportationProvinceCapitalToLimaMarket.csv')

In [27]:
createNumberTimeVariable(provinceCapitalToLimaMarket, "DistanceInTime", "provinceCapitalToLimaMarket")

In [28]:
provinceCapitalToLimaMarket

,NOMBREPV,NOMBREDD,DistanceInTime,DistanceInKm,provinceCapitalToLimaMarket
0,CHACHAPOYAS,AMAZONAS,20 h 49 min,1230 km,20.8
1,HUARAZ,ANCASH,6 h 56 min,412 km,6.93
2,APURÍMAC,APURÍMAC,18 h 7 min,985 km,18.1
3,AREQUIPA,AREQUIPA,15 h 42 min,1008 km,15.7
4,AYACUCHO,AYACUCHO,8 h 25 min,553 km,8.42
5,CAJAMARCA,CAJAMARCA,15 h 12 min,869 km,15.2
6,CUSCO,CUSCO,20 h 24 min,1093 km,20.4
7,HUANCAVELICA,HUANCAVELICA,6 h 55 min,419 km,6.92
8,HUÁNUCO,HUÁNUCO,8 h 40 min,407 km,8.67
9,ICA,ICA,3 h 59 min,297 km,3.98


In [29]:
provinceCapitalToLimaMarket.to_csv('2019_04RoadTransportationProvinceCapitalToLimaMarket.csv')